# Data Preparation

In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import optuna.integration.lightgbm as lgb          ####
from lightgbm import LGBMRegressor, plot_metric    #### 여기서 lgb로 안가져옴
from pathlib import Path
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, max_error, mean_absolute_error, mean_squared_log_error, mean_absolute_percentage_error, median_absolute_error

In [2]:
# options
pd.set_option('max_columns',100)
plt.style.use('fivethirtyeight')
warnings.simplefilter('ignore')
seed = 1

In [3]:
# Data dirctory
data_dir = Path('../eda/')
data_file = data_dir / 'Data.csv'

In [27]:
# Data
# 0:AAR / 1:EAD / 2:ADR / 3:EDD는 고정  , 나머지는 순서 상관 없음
Data = pd.read_csv(data_file, index_col=0)

***
***
***

# Data Selection

In [28]:
# 필요없는 것을 버리기
Data_temp = Data.drop('TMP', axis=1)
Data_temp = Data_temp.drop('TD', axis=1)
Data_temp = Data_temp.drop('HM', axis=1)
Data_temp = Data_temp.drop('PS', axis=1)
Data_temp = Data_temp.drop('PA', axis=1)

#고층바람 너무 높은 고도는 뺴자 
Data_temp = Data_temp.drop('WD_400', axis=1)
Data_temp = Data_temp.drop('WD_500', axis=1)
Data_temp = Data_temp.drop('WD_700', axis=1)
Data_temp = Data_temp.drop('WS_400', axis=1)
Data_temp = Data_temp.drop('WS_500', axis=1)
Data_temp = Data_temp.drop('WS_700', axis=1)

# drop TAF
for i in range(6,30,6):
    Data_temp = Data_temp.drop(f'WDIR_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'WSPD_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'WG_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'VIS_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'WC_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'CLA_1LYR_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'BASE_1LYR_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'CLA_2LYR_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'BASE_2LYR_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'CLA_3LYR_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'BASE_3LYR_t{i}', axis=1)

In [29]:
# 각 시간에 맞는 TAF로 나누기
taf6 = [12,18,24]
taf12 = [6,18,24]
taf18 = [6,12,24]
taf24 = [6,12,18]
    
# 각 시간에 맞는 taf 넣기
data_taf = {}
for i in range(6,30,6):
    data_taf[f'Data_{i}'] = Data_temp    
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'WDIR_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'WSPD_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'WG_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'VIS_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'WC_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'CLA_1LYR_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'BASE_1LYR_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'CLA_2LYR_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'BASE_2LYR_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'CLA_3LYR_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'BASE_3LYR_t{i}'])
    
Data_6 = data_taf['Data_6']
Data_12 = data_taf['Data_12']
Data_18 = data_taf['Data_18']
Data_24 = data_taf['Data_24']

Validation <br><br>

sklearn.model_selection
* train_test_split
* KFold
* StratifiedKFold

<br>
<br>
lightGBM <br>
* lgb.cv    # 쓰려면 lightgbm을 lgb로 import

***
***
***

# LightGBM fitting

## Data Split

In [30]:
# 예측할 시간에 맞는 Data로 넣기
# 0-6 : Data_6 / 6-12 : Data_12 / 12-18 : Data_18 / 18-24 : Data_24
Data_raw = Data_6
Data_m = Data_6
Data_m = Data_m.drop('AAR', axis=1)
Data_m = Data_m.drop('ADR', axis=1)


# Arrival
y_a = Data_raw.AAR.to_numpy()
X_a = Data_m.to_numpy()
#X_train_a, X_test_a, y_train_a, y_test_a = train_test_split(X_a, y_a, test_size = 0.1, random_state = seed)
X_train_a, X_test_a, y_train_a, y_test_a = train_test_split(X_a, y_a, test_size = 0.1, random_state = seed)
X_train_a, X_val_a, y_train_a, y_val_a = train_test_split(X_train_a, y_train_a, test_size=0.3, random_state = 13) 


# Departure
y_d = Data_raw.ADR.to_numpy()
X_d = Data_m.to_numpy()
#X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_d, y_d, test_size = 0.1, random_state = seed)
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_d, y_d, test_size = 0.1, random_state = seed)
X_train_d, X_val_d, y_train_d, y_val_d = train_test_split(X_train_d, y_train_d, test_size=0.3, random_state = 13) 

# val은 hyperparameter 검증에 사용
# 0.11 x 0.9 = 0.099

print('Training Data shape : ', Data_m.shape)
Data_m.head()

Training Data shape :  (8760, 46)


,EAD,EDD,year,month,day,hour,DayName,Arpt_cond,P_Airp,P_AAR,P_ADR,Arrival_remainder,Departure_remainder,WD_850,WD_925,WD_1000,WS_850,WS_925,WS_1000,WD,WSPD,WS_GST,VIS,WC,RN,CA_TOT,CLA_1LYR,BASE_1LYR,CLA_2LYR,BASE_2LYR,CLA_3LYR,BASE_3LYR,CLA_4LYR,BASE_4LYR,RVR,WDIR_t6,WSPD_t6,WG_t6,VIS_t6,WC_t6,CLA_1LYR_t6,BASE_1LYR_t6,CLA_2LYR_t6,BASE_2LYR_t6,CLA_3LYR_t6,BASE_3LYR_t6
0,3.0,9.0,2019,1,1,0,2,1.0,1.0,0.0,0.0,0.0,0.0,280.0,280.0,305.0,12.0,23.0,8.0,1,6,0.0,1000,1,0.0,0,0.0,400.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0
1,3.0,7.0,2019,1,1,1,2,1.0,1.0,4.0,7.0,0.0,2.0,280.0,280.0,305.0,12.0,23.0,8.0,31,5,0.0,1000,1,0.0,0,0.0,400.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0
2,2.0,3.0,2019,1,1,2,2,1.0,1.0,2.0,6.0,1.0,1.0,280.0,280.0,305.0,12.0,23.0,8.0,33,4,0.0,1000,1,0.0,0,0.0,400.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0
3,1.0,2.0,2019,1,1,3,2,1.0,1.0,0.0,3.0,2.0,0.0,280.0,280.0,305.0,12.0,23.0,8.0,32,4,0.0,1000,1,0.0,0,0.0,400.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0
4,15.0,2.0,2019,1,1,4,2,1.0,1.0,6.0,5.0,0.0,0.0,280.0,280.0,305.0,12.0,23.0,8.0,7,1,0.0,1000,1,0.0,2,2.0,30.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0


***
## Hyperparameter Optimization

In [31]:
# Hyperparameters
params = {'boosting_type' : 'gbdt',            # 'dart' 는 계산시간 길어짐, early stopping X / 'rf’ : Random Forest
          'metric': 'mse',
          'num_leaves' : 127,                  ## Maximum tree leaves for base learners (31)
          'max_depth' : - 1,                   # Maximum tree depth for base learners, <=0 means no limit (-1)
          'learning_rate' : 0.001,             ## Boosting learning rate (0.1)
          'n_estimators' : 10000000,           # Number of boosted trees to fit (100) -> fit에서 early stopping으로 제한해서 크게 설정함
          'subsample_for_bin' : 200000,        # Number of samples for constructing bins (200000)
          'objective' : 'regression',          # learning task and the corresponding learning objective (None)
          'class_weight' : None,               # * Use this parameter only for multi-class classification task
          'min_split_gain' : 0.0,              # Minimum loss reduction required to make a further partition on a leaf node of the tree (0)
          'min_child_weight' : 0.001,          # Minimum sum of instance weight (hessian) needed in a child (leaf) (0.001)
          'min_child_samples' : 1,             # Minimum number of data needed in a child (leaf) (20) - 마지막노드(리프)에 최소 몇가지 샘플이 있어야 하는지 
          'subsample' : 0.8,                   ## Subsample ratio of the training instance (1.0) - 개별 트리를 학습시키는데 몇 %의 데이터를 사용할 것 인지, row sampling
          'subsample_freq' : 1,                # Frequency of subsample, <=0 means no enable (0) - 몇개의 트리마다 subsampling을 할 것인지
          'colsample_bytree' : 0.8,            ## Subsample ratio of columns when constructing each tree (1.0) - 몇 %의 column을 sampling 할 것인지
          'reg_alpha' : 0.0,                   # L1 regularization term on weights (0)
          'reg_lambda' : 0.0,                  # L2 regularization term on weights (0)
          'random_state' : seed,               # Random number seed (None)
          'n_jobs' : - 1,                      # Number of parallel threads (-1) - 몇 개의 병렬작업을 할 것인지 (-1 = 모든 가능한 것 전부)
          'silent' : True,                     # Whether to print messages while running boosting (True)
          'importance_type' : 'split'}         # ‘split’: result contains numbers of times the feature is used in a model
                                               # ‘gain’ : result contains total gains of splits which use the feature

# 최적 hyperparameter 찾기!!

In [32]:
# Hyperparameters - arrival
params_a = {'boosting_type' : 'gbdt',                   # 'dart' 는 계산시간 길어짐, early stopping X / 'rf’ : Random Forest
            'metric': 'mse',
            'num_leaves' : 127,                          ## Maximum tree leaves for base learners (31)
            'max_depth' : - 1,                          # Maximum tree depth for base learners, <=0 means no limit (-1)
            'learning_rate' : 0.001,                    ## Boosting learning rate (0.1)
            'n_estimators' : 10000000,                  # Number of boosted trees to fit (100) -> fit에서 early stopping으로 제한해서 크게 설정함
            'subsample_for_bin' : 200000,               # Number of samples for constructing bins (200000)
            'objective' : 'regression',                 # learning task and the corresponding learning objective (None)
            'class_weight' : None,                      # * Use this parameter only for multi-class classification task
            'min_split_gain' : 0.0,                     # Minimum loss reduction required to make a further partition on a leaf node of the tree (0)
            'min_child_weight' : 0.001,                 # Minimum sum of instance weight (hessian) needed in a child (leaf) (0.001)
            'min_child_samples' : 1,                    # Minimum number of data needed in a child (leaf) (20) - 마지막노드(리프)에 최소 몇가지 샘플이 있어야 하는지 
            'feature_pre_filter': False,
            'subsample' : 0.8,           ## Subsample ratio of the training instance (1.0) - 개별 트리를 학습시키는데 몇 %의 데이터를 사용할 것 인지, row sampling
            'subsample_freq' : 1,   #3                  # Frequency of subsample, <=0 means no enable (0) - 몇개의 트리마다 subsampling을 할 것인지
            'colsample_bytree' : 0.6839999999999999,    ## Subsample ratio of columns when constructing each tree (1.0) - 몇 %의 column을 sampling 할 것인지
            'reg_alpha' : 1.5913347234314888e-05,       # L1 regularization term on weights (0)
            'reg_lambda' : 0.8083105161094011,           # L2 regularization term on weights (0)
            'random_state' : seed,                      # Random number seed (None)
            'n_jobs' : - 1,                             # Number of parallel threads (-1) - 몇 개의 병렬작업을 할 것인지 (-1 = 모든 가능한 것 전부)
            'silent' : True,                            # Whether to print messages while running boosting (True)
            'importance_type' : 'split'}                # ‘split’: result contains numbers of times the feature is used in a model
                                                        # ‘gain’ : result contains total gains of splits which use the feature

In [33]:
# Optuna 간단 버전 - arrival

"""
y = Data_raw.AAR.to_numpy()
X = Data_m.to_numpy()

X_train_a, X_val_a, y_train_a, y_val_a = train_test_split(X, y, test_size=.2, random_state=seed)

dtrain_a = lgb.Dataset(X_train_a, label = y_train_a)
dval_a = lgb.Dataset(X_val_a, label = y_val_a)

model = lgb.train(params, dtrain_a,
                  valid_sets=[dtrain_a, dval_a], 
                  verbose_eval=100,
                  early_stopping_rounds=10)

prediction = model.predict(X_val_a, num_iteration=model.best_iteration)

accuracy = r2_score(y_val_a, prediction)        # classification이면 앞에 argmax( , axis = 1)로 하면 될 듯



params = model.params
print("Best params:", params)
print("  Accuracy = {}".format(accuracy))
print("  Params: ")
for key, value in params.items():
    print("    {}: {}".format(key, value))

params_a = params
"""

'\ny = Data_raw.AAR.to_numpy()\nX = Data_m.to_numpy()\n\nX_train_a, X_val_a, y_train_a, y_val_a = train_test_split(X, y, test_size=.2, random_state=seed)\n\ndtrain_a = lgb.Dataset(X_train_a, label = y_train_a)\ndval_a = lgb.Dataset(X_val_a, label = y_val_a)\n\nmodel = lgb.train(params, dtrain_a,\n                  valid_sets=[dtrain_a, dval_a], \n                  verbose_eval=100,\n                  early_stopping_rounds=10)\n\nprediction = model.predict(X_val_a, num_iteration=model.best_iteration)\n\naccuracy = r2_score(y_val_a, prediction)        # classification이면 앞에 argmax( , axis = 1)로 하면 될 듯\n\n\n\nparams = model.params\nprint("Best params:", params)\nprint("  Accuracy = {}".format(accuracy))\nprint("  Params: ")\nfor key, value in params.items():\n    print("    {}: {}".format(key, value))\n\nparams_a = params\n'

In [34]:
# Hyperparameters - departure
params_d = {'boosting_type' : 'gbdt',                 # 'dart' 는 계산시간 길어짐, early stopping X / 'rf’ : Random Forest
            'metric': 'mse',
            'num_leaves' : 127,                        ## Maximum tree leaves for base learners (31)
            'max_depth' : - 1,                        # Maximum tree depth for base learners, <=0 means no limit (-1)
            'learning_rate' : 0.001,                  ## Boosting learning rate (0.1)
            'n_estimators' : 10000000,                # Number of boosted trees to fit (100) -> fit에서 early stopping으로 제한해서 크게 설정함
            'subsample_for_bin' : 200000,             # Number of samples for constructing bins (200000)
            'objective' : 'regression',               # learning task and the corresponding learning objective (None)
            'class_weight' : None,                    # * Use this parameter only for multi-class classification task
            'min_split_gain' : 0.0,                   # Minimum loss reduction required to make a further partition on a leaf node of the tree (0)
            'min_child_weight' : 0.001,               # Minimum sum of instance weight (hessian) needed in a child (leaf) (0.001)
            'min_child_samples' : 1,                  # Minimum number of data needed in a child (leaf) (20) - 마지막노드(리프)에 최소 몇가지 샘플이 있어야 하는지 
            'feature_pre_filter': False,
            'subsample' : 0.9220698151647799,         ## Subsample ratio of the training instance (1.0) - 개별 트리를 학습시키는데 몇 %의 데이터를 사용할 것 인지, row sampling
            'subsample_freq' : 1,    #3               # Frequency of subsample, <=0 means no enable (0) - 몇개의 트리마다 subsampling을 할 것인지
            'colsample_bytree' : 0.8480000000000001,  ## Subsample ratio of columns when constructing each tree (1.0) - 몇 %의 column을 sampling 할 것인지
            'reg_alpha' : 1.2560334090582146e-07,     # L1 regularization term on weights (0)
            'reg_lambda' : 0.003315287591858434,      # L2 regularization term on weights (0)
            'random_state' : seed,                    # Random number seed (None)
            'n_jobs' : - 1,                           # Number of parallel threads (-1) - 몇 개의 병렬작업을 할 것인지 (-1 = 모든 가능한 것 전부)
            'silent' : True,                          # Whether to print messages while running boosting (True)
            'importance_type' : 'split'}              # ‘split’: result contains numbers of times the feature is used in a model
                                                      # ‘gain’ : result contains total gains of splits which use the feature

In [35]:
# Optuna 간단 버전 - departure

"""
y = Data_raw.ADR.to_numpy()
X = Data_m.to_numpy()

X_train_d, X_val_d, y_train_d, y_val_d = train_test_split(X, y, test_size=.2, random_state=seed)

dtrain_d = lgb.Dataset(X_train_d, label = y_train_d)
dval_d = lgb.Dataset(X_val_d, label = y_val_d)

model = lgb.train(params, dtrain_d,
                  valid_sets=[dtrain_d, dval_d], 
                  verbose_eval=100,
                  early_stopping_rounds=10)

prediction = model.predict(X_val_d, num_iteration=model.best_iteration)

accuracy = r2_score(y_val_d, prediction)         # classification이면 앞에 argmax( , axis = 1)로 하면 될 듯

params = model.params
print("Best params:", params)
print("  Accuracy = {}".format(accuracy))
print("  Params: ")
for key, value in params.items():
    print("    {}: {}".format(key, value))

params_d = params
"""

'\ny = Data_raw.ADR.to_numpy()\nX = Data_m.to_numpy()\n\nX_train_d, X_val_d, y_train_d, y_val_d = train_test_split(X, y, test_size=.2, random_state=seed)\n\ndtrain_d = lgb.Dataset(X_train_d, label = y_train_d)\ndval_d = lgb.Dataset(X_val_d, label = y_val_d)\n\nmodel = lgb.train(params, dtrain_d,\n                  valid_sets=[dtrain_d, dval_d], \n                  verbose_eval=100,\n                  early_stopping_rounds=10)\n\nprediction = model.predict(X_val_d, num_iteration=model.best_iteration)\n\naccuracy = r2_score(y_val_d, prediction)         # classification이면 앞에 argmax( , axis = 1)로 하면 될 듯\n\nparams = model.params\nprint("Best params:", params)\nprint("  Accuracy = {}".format(accuracy))\nprint("  Params: ")\nfor key, value in params.items():\n    print("    {}: {}".format(key, value))\n\nparams_d = params\n'

***
## Arrival

In [13]:
# Model fitting
reg_arrival = LGBMRegressor(**params_a)

reg_arrival.fit(X_train_a, y_train_a,
                sample_weight = None,                   # Weights of training data
                init_score = None,                      # Weights of training data
                eval_set = [(X_val_a, y_val_a)],        # pairs to use as validation sets
                eval_sample_weight = None,              # Weights of eval data
                eval_init_score = None,                 # Init score of eval data.   
                eval_metric = 'l2',                     # Default: ‘l2’ for LGBMRegressor, ‘logloss’ for LGBMClassifier
                early_stopping_rounds = 10,             # loss fuction이 n번 이상 좋아지지 않으면 멈춰라
                verbose = True)            

[1]	valid_0's l2: 127.986
Training until validation scores don't improve for 10 rounds
[2]	valid_0's l2: 127.749
[3]	valid_0's l2: 127.519
[4]	valid_0's l2: 127.286
[5]	valid_0's l2: 127.055
[6]	valid_0's l2: 126.82
[7]	valid_0's l2: 126.592
[8]	valid_0's l2: 126.363
[9]	valid_0's l2: 126.131
[10]	valid_0's l2: 125.898
[11]	valid_0's l2: 125.668
[12]	valid_0's l2: 125.436
[13]	valid_0's l2: 125.205
[14]	valid_0's l2: 124.977
[15]	valid_0's l2: 124.746
[16]	valid_0's l2: 124.52
[17]	valid_0's l2: 124.296
[18]	valid_0's l2: 124.066
[19]	valid_0's l2: 123.843
[20]	valid_0's l2: 123.619
[21]	valid_0's l2: 123.393
[22]	valid_0's l2: 123.17
[23]	valid_0's l2: 122.98
[24]	valid_0's l2: 122.755
[25]	valid_0's l2: 122.529
[26]	valid_0's l2: 122.306
[27]	valid_0's l2: 122.081
[28]	valid_0's l2: 121.881
[29]	valid_0's l2: 121.657
[30]	valid_0's l2: 121.434
[31]	valid_0's l2: 121.212
[32]	valid_0's l2: 120.99
[33]	valid_0's l2: 120.77
[34]	valid_0's l2: 120.552
[35]	valid_0's l2: 120.331
[36]	vali

[347]	valid_0's l2: 70.1003
[348]	valid_0's l2: 69.9809
[349]	valid_0's l2: 69.8612
[350]	valid_0's l2: 69.7427
[351]	valid_0's l2: 69.6225
[352]	valid_0's l2: 69.5035
[353]	valid_0's l2: 69.3841
[354]	valid_0's l2: 69.2659
[355]	valid_0's l2: 69.1473
[356]	valid_0's l2: 69.0278
[357]	valid_0's l2: 68.9108
[358]	valid_0's l2: 68.7916
[359]	valid_0's l2: 68.6716
[360]	valid_0's l2: 68.5558
[361]	valid_0's l2: 68.4393
[362]	valid_0's l2: 68.3214
[363]	valid_0's l2: 68.2052
[364]	valid_0's l2: 68.0901
[365]	valid_0's l2: 67.9726
[366]	valid_0's l2: 67.8586
[367]	valid_0's l2: 67.7419
[368]	valid_0's l2: 67.6275
[369]	valid_0's l2: 67.5134
[370]	valid_0's l2: 67.3994
[371]	valid_0's l2: 67.3163
[372]	valid_0's l2: 67.2006
[373]	valid_0's l2: 67.0854
[374]	valid_0's l2: 66.9751
[375]	valid_0's l2: 66.8954
[376]	valid_0's l2: 66.783
[377]	valid_0's l2: 66.6694
[378]	valid_0's l2: 66.557
[379]	valid_0's l2: 66.4434
[380]	valid_0's l2: 66.3289
[381]	valid_0's l2: 66.2184
[382]	valid_0's l2: 66

[681]	valid_0's l2: 41.0553
[682]	valid_0's l2: 40.9922
[683]	valid_0's l2: 40.9288
[684]	valid_0's l2: 40.866
[685]	valid_0's l2: 40.8031
[686]	valid_0's l2: 40.7406
[687]	valid_0's l2: 40.6779
[688]	valid_0's l2: 40.6164
[689]	valid_0's l2: 40.554
[690]	valid_0's l2: 40.4906
[691]	valid_0's l2: 40.4286
[692]	valid_0's l2: 40.3664
[693]	valid_0's l2: 40.3034
[694]	valid_0's l2: 40.241
[695]	valid_0's l2: 40.1786
[696]	valid_0's l2: 40.1151
[697]	valid_0's l2: 40.0539
[698]	valid_0's l2: 39.9931
[699]	valid_0's l2: 39.931
[700]	valid_0's l2: 39.8692
[701]	valid_0's l2: 39.8085
[702]	valid_0's l2: 39.7465
[703]	valid_0's l2: 39.6846
[704]	valid_0's l2: 39.6242
[705]	valid_0's l2: 39.5636
[706]	valid_0's l2: 39.5019
[707]	valid_0's l2: 39.4419
[708]	valid_0's l2: 39.3811
[709]	valid_0's l2: 39.3198
[710]	valid_0's l2: 39.2594
[711]	valid_0's l2: 39.199
[712]	valid_0's l2: 39.1382
[713]	valid_0's l2: 39.0849
[714]	valid_0's l2: 39.0239
[715]	valid_0's l2: 38.9641
[716]	valid_0's l2: 38.90

[981]	valid_0's l2: 26.7997
[982]	valid_0's l2: 26.763
[983]	valid_0's l2: 26.7284
[984]	valid_0's l2: 26.6925
[985]	valid_0's l2: 26.6573
[986]	valid_0's l2: 26.6215
[987]	valid_0's l2: 26.5865
[988]	valid_0's l2: 26.552
[989]	valid_0's l2: 26.5165
[990]	valid_0's l2: 26.4812
[991]	valid_0's l2: 26.446
[992]	valid_0's l2: 26.4152
[993]	valid_0's l2: 26.3798
[994]	valid_0's l2: 26.3448
[995]	valid_0's l2: 26.3089
[996]	valid_0's l2: 26.2752
[997]	valid_0's l2: 26.2402
[998]	valid_0's l2: 26.2055
[999]	valid_0's l2: 26.1705
[1000]	valid_0's l2: 26.1354
[1001]	valid_0's l2: 26.1008
[1002]	valid_0's l2: 26.0663
[1003]	valid_0's l2: 26.0311
[1004]	valid_0's l2: 25.9962
[1005]	valid_0's l2: 25.9619
[1006]	valid_0's l2: 25.9273
[1007]	valid_0's l2: 25.8927
[1008]	valid_0's l2: 25.8584
[1009]	valid_0's l2: 25.8245
[1010]	valid_0's l2: 25.7903
[1011]	valid_0's l2: 25.756
[1012]	valid_0's l2: 25.7228
[1013]	valid_0's l2: 25.6884
[1014]	valid_0's l2: 25.6729
[1015]	valid_0's l2: 25.6406
[1016]	v

[1269]	valid_0's l2: 18.9697
[1270]	valid_0's l2: 18.9487
[1271]	valid_0's l2: 18.9283
[1272]	valid_0's l2: 18.9075
[1273]	valid_0's l2: 18.8865
[1274]	valid_0's l2: 18.8657
[1275]	valid_0's l2: 18.8452
[1276]	valid_0's l2: 18.8253
[1277]	valid_0's l2: 18.8044
[1278]	valid_0's l2: 18.7838
[1279]	valid_0's l2: 18.7635
[1280]	valid_0's l2: 18.7435
[1281]	valid_0's l2: 18.7355
[1282]	valid_0's l2: 18.7148
[1283]	valid_0's l2: 18.6939
[1284]	valid_0's l2: 18.6738
[1285]	valid_0's l2: 18.6533
[1286]	valid_0's l2: 18.6333
[1287]	valid_0's l2: 18.6135
[1288]	valid_0's l2: 18.5929
[1289]	valid_0's l2: 18.5729
[1290]	valid_0's l2: 18.5526
[1291]	valid_0's l2: 18.5325
[1292]	valid_0's l2: 18.5188
[1293]	valid_0's l2: 18.4991
[1294]	valid_0's l2: 18.479
[1295]	valid_0's l2: 18.4591
[1296]	valid_0's l2: 18.4389
[1297]	valid_0's l2: 18.4235
[1298]	valid_0's l2: 18.4039
[1299]	valid_0's l2: 18.3844
[1300]	valid_0's l2: 18.3645
[1301]	valid_0's l2: 18.3448
[1302]	valid_0's l2: 18.3255
[1303]	valid_0'

...

[3935]	valid_0's l2: 7.76925
[3936]	valid_0's l2: 7.76907
[3937]	valid_0's l2: 7.76879
[3938]	valid_0's l2: 7.76853
[3939]	valid_0's l2: 7.76821
[3940]	valid_0's l2: 7.76804
[3941]	valid_0's l2: 7.76787
[3942]	valid_0's l2: 7.76773
[3943]	valid_0's l2: 7.76758
[3944]	valid_0's l2: 7.7672
[3945]	valid_0's l2: 7.76688
[3946]	valid_0's l2: 7.76662
[3947]	valid_0's l2: 7.76647
[3948]	valid_0's l2: 7.76625
[3949]	valid_0's l2: 7.76618
[3950]	valid_0's l2: 7.76576
[3951]	valid_0's l2: 7.76548
[3952]	valid_0's l2: 7.76519
[3953]	valid_0's l2: 7.76495
[3954]	valid_0's l2: 7.76466
[3955]	valid_0's l2: 7.76456
[3956]	valid_0's l2: 7.76427
[3957]	valid_0's l2: 7.76404
[3958]	valid_0's l2: 7.76391
[3959]	valid_0's l2: 7.76371
[3960]	valid_0's l2: 7.76362
[3961]	valid_0's l2: 7.76319
[3962]	valid_0's l2: 7.76306
[3963]	valid_0's l2: 7.7629
[3964]	valid_0's l2: 7.76277
[3965]	valid_0's l2: 7.76249
[3966]	valid_0's l2: 7.76223
[3967]	valid_0's l2: 7.76203
[3968]	valid_0's l2: 7.76188
[3969]	valid_0's

[4239]	valid_0's l2: 7.71607
[4240]	valid_0's l2: 7.71587
[4241]	valid_0's l2: 7.7159
[4242]	valid_0's l2: 7.7158
[4243]	valid_0's l2: 7.7157
[4244]	valid_0's l2: 7.71558
[4245]	valid_0's l2: 7.71543
[4246]	valid_0's l2: 7.71523
[4247]	valid_0's l2: 7.71513
[4248]	valid_0's l2: 7.71483
[4249]	valid_0's l2: 7.71476
[4250]	valid_0's l2: 7.71477
[4251]	valid_0's l2: 7.71448
[4252]	valid_0's l2: 7.71445
[4253]	valid_0's l2: 7.71423
[4254]	valid_0's l2: 7.71404
[4255]	valid_0's l2: 7.71391
[4256]	valid_0's l2: 7.71379
[4257]	valid_0's l2: 7.7136
[4258]	valid_0's l2: 7.71356
[4259]	valid_0's l2: 7.71329
[4260]	valid_0's l2: 7.71317
[4261]	valid_0's l2: 7.71293
[4262]	valid_0's l2: 7.71274
[4263]	valid_0's l2: 7.71271
[4264]	valid_0's l2: 7.7124
[4265]	valid_0's l2: 7.71221
[4266]	valid_0's l2: 7.71221
[4267]	valid_0's l2: 7.712
[4268]	valid_0's l2: 7.71197
[4269]	valid_0's l2: 7.71184
[4270]	valid_0's l2: 7.71176
[4271]	valid_0's l2: 7.71171
[4272]	valid_0's l2: 7.71151
[4273]	valid_0's l2: 

[4544]	valid_0's l2: 7.68318
[4545]	valid_0's l2: 7.68313
[4546]	valid_0's l2: 7.68293
[4547]	valid_0's l2: 7.68286
[4548]	valid_0's l2: 7.68275
[4549]	valid_0's l2: 7.68257
[4550]	valid_0's l2: 7.68257
[4551]	valid_0's l2: 7.68239
[4552]	valid_0's l2: 7.68237
[4553]	valid_0's l2: 7.68219
[4554]	valid_0's l2: 7.68226
[4555]	valid_0's l2: 7.68218
[4556]	valid_0's l2: 7.68213
[4557]	valid_0's l2: 7.68194
[4558]	valid_0's l2: 7.6817
[4559]	valid_0's l2: 7.68155
[4560]	valid_0's l2: 7.68151
[4561]	valid_0's l2: 7.68147
[4562]	valid_0's l2: 7.68126
[4563]	valid_0's l2: 7.68112
[4564]	valid_0's l2: 7.68105
[4565]	valid_0's l2: 7.68098
[4566]	valid_0's l2: 7.68091
[4567]	valid_0's l2: 7.68088
[4568]	valid_0's l2: 7.68089
[4569]	valid_0's l2: 7.68074
[4570]	valid_0's l2: 7.68063
[4571]	valid_0's l2: 7.68039
[4572]	valid_0's l2: 7.68046
[4573]	valid_0's l2: 7.68047
[4574]	valid_0's l2: 7.68049
[4575]	valid_0's l2: 7.68054
[4576]	valid_0's l2: 7.68045
[4577]	valid_0's l2: 7.68018
[4578]	valid_0'

[4842]	valid_0's l2: 7.66107
[4843]	valid_0's l2: 7.66097
[4844]	valid_0's l2: 7.66087
[4845]	valid_0's l2: 7.66093
[4846]	valid_0's l2: 7.66081
[4847]	valid_0's l2: 7.6608
[4848]	valid_0's l2: 7.66072
[4849]	valid_0's l2: 7.66071
[4850]	valid_0's l2: 7.66068
[4851]	valid_0's l2: 7.66077
[4852]	valid_0's l2: 7.66073
[4853]	valid_0's l2: 7.66069
[4854]	valid_0's l2: 7.66069
[4855]	valid_0's l2: 7.66048
[4856]	valid_0's l2: 7.66045
[4857]	valid_0's l2: 7.66034
[4858]	valid_0's l2: 7.66037
[4859]	valid_0's l2: 7.66042
[4860]	valid_0's l2: 7.66025
[4861]	valid_0's l2: 7.66026
[4862]	valid_0's l2: 7.66026
[4863]	valid_0's l2: 7.66027
[4864]	valid_0's l2: 7.66025
[4865]	valid_0's l2: 7.66023
[4866]	valid_0's l2: 7.66019
[4867]	valid_0's l2: 7.66007
[4868]	valid_0's l2: 7.65999
[4869]	valid_0's l2: 7.65999
[4870]	valid_0's l2: 7.66007
[4871]	valid_0's l2: 7.65999
[4872]	valid_0's l2: 7.66003
[4873]	valid_0's l2: 7.66013
[4874]	valid_0's l2: 7.66
[4875]	valid_0's l2: 7.65991
[4876]	valid_0's l

LGBMRegressor(colsample_bytree=0.6839999999999999, feature_pre_filter=False,
              learning_rate=0.001, metric='mse', min_child_samples=1,
              n_estimators=10000000, num_leaves=127, objective='regression',
              random_state=1, reg_alpha=1.5913347234314888e-05,
              reg_lambda=0.8083105161094011, subsample=0.8, subsample_freq=1)

***
##  Departure

In [14]:
# Model fitting
reg_departure = LGBMRegressor(**params_d)

reg_departure.fit(X_train_d, y_train_d,
                  eval_set=[(X_val_d, y_val_d)],
                  eval_metric='l2',
                  early_stopping_rounds = 10, 
                  verbose = False)     # loss fuction이 n번 이상 좋아지지 않으면 멈춰라

LGBMRegressor(colsample_bytree=0.8480000000000001, feature_pre_filter=False,
              learning_rate=0.001, metric='mse', min_child_samples=1,
              n_estimators=10000000, num_leaves=127, objective='regression',
              random_state=1, reg_alpha=1.2560334090582146e-07,
              reg_lambda=0.003315287591858434, subsample=0.9220698151647799,
              subsample_freq=1)

***

## Cross validation

for loop안에서 predict하고 평균을 내야 함 <br>
-> demand를 하나씩 증가시키리면, 바로 CV를 사용하기는 어려울 듯

In [ ]:
"""
n_fold = 10

cv = KFold(n_splits=n_fold, shuffle=True, random_state=seed)

p_val_a = np.zeros(X_a.shape[0])
predict_a = np.zeros(X_a.shape[0])
p_val_d = np.zeros(X_d.shape[0])
predict_d = np.zeros(X_d.shape[0])

############### 위에서 X_val_a등으로 바꾼 거 없애고 -> 아래 X_a, y_a, X_d, y_d를 X_train_a 등으로 바꾸기 #######################

for i, (i_trn, i_val) in enumerate(cv.split(X_a, y_a), 1):        # 몇번째인지 보기 위해 enumerate 사용
    print('----------------------------------------------------------------------------')
    print(f'Training model for Cross-Validation #{i}')
    reg_arrival = LGBMRegressor(**params)    #############params_a
    reg_arrival.fit(X_a[i_trn], y_a[i_trn],
                sample_weight = None,                   # Weights of training data
                init_score = None,                      # Weights of training data
                eval_set = [(X_a[i_val], y_a[i_val])],  # pairs to use as validation sets
                eval_sample_weight = None,              # Weights of eval data
                eval_init_score = None,                 # Init score of eval data.   
                eval_metric = 'l2',                     # Default: ‘l2’ for LGBMRegressor, ‘logloss’ for LGBMClassifier, ‘ndcg’ for LGBMRanker
                early_stopping_rounds = 10 )            # loss fuction이 n번 이상 좋아지지 않으면 멈춰라
    
    p_val_a[i_val] = reg_arrival.predict(X_a[i_val])
    predict_a += reg_arrival.predict(X_a) / n_fold      ##### 이 자리에 원래는 test data가 들어가면 됨
                                                        # test data의 prediction은 CV에서 각 dataset이 예측한 값들의 평균이므로
    
for i, (i_trn, i_val) in enumerate(cv.split(X_d, y_d), 1):        # 몇번째인지 보기 위해 enumerate 사용
    print('----------------------------------------------------------------------------')
    print(f'Training model for Cross-Validation #{i}')
    reg_departure = LGBMRegressor(**params)    #############params_d
    reg_departure.fit(X_d[i_trn], y_d[i_trn],
                      sample_weight = None,                   # Weights of training data
                      init_score = None,                      # Weights of training data
                      eval_set = [(X_d[i_val], y_d[i_val])],  # pairs to use as validation sets
                      eval_sample_weight = None,              # Weights of eval data
                      eval_init_score = None,                 # Init score of eval data.   
                      eval_metric = 'l2',                     # Default: ‘l2’ for LGBMRegressor, ‘logloss’ for LGBMClassifier, ‘ndcg’ for LGBMRanker
                      early_stopping_rounds = 10 )            # loss fuction이 n번 이상 좋아지지 않으면 멈춰라
    
    p_val_d[i_val] = reg_departure.predict(X_d[i_val])
    predict_d += reg_departure.predict(X_d) / n_fold          ##### 이 자리에 원래는 test data가 들어가면 됨
                                                              # test data의 prediction은 CV에서 각 dataset이 예측한 값들의 평균이므로
"""

In [ ]:
"""
print(f'Arrival RMSE : {np.sqrt(mean_squared_error(y_a, predict_a)):.4f}')
print(f'Arrival Training R^2 : {r2_score(y_a, predict_a) * 100:.4f}')
print(f'Departure RMSE : {np.sqrt(mean_squared_error(y_d, predict_d)):.4f}')
print(f'Departure Training R^2 : {r2_score(y_d, predict_d) * 100:.4f}')

np.savetxt('p_val_a.csv', p_val_a, fmt='%.6f', delimiter=',')
np.savetxt('p_val_d.csv', p_val_d, fmt='%.6f', delimiter=',')
np.savetxt('predict_a.csv', predict_a, fmt='%.6f', delimiter=',')
np.savetxt('predict_d.csv', predict_d, fmt='%.6f', delimiter=',')
"""

이렇게 각각의 model의(e.g. lightGBM, XGBOOST, Randomforest ...) Cross Validation을 예측한 결과와, Test data에 대해 예측한 결과를 파일로 저장 <br>
-> 다음 stage에서는 CV결과들을 input data로 사용, 기존의 label은 그대로 사용, test data의 예측값을 다음 stage에서 test data의 input으로 사용 <br>
-> 계속 쌓아갈 수 있음 (=Stacking, 보통 stage 1,2정도면 이후 효과는 미비)


stacking, ensemble 보다 feature engineering, hyperparameter tuning의 성능 향상이 훨씬 커서 앞의 방법은 굳이 사용하지 않아도 괜찮음

***
***
***

# Evaluation

<sklearn.metrics> <br>

* regression <br>
mean_squared_error, mean_absolute_error, r2_score <br><br>

* classification <br>
log_loss, roc_auc_score, accuracy_score, confusion_matrix <br>

In [15]:
def Test(Data_raw, ops='arrival', start=0, end = 10):        # ops : arrival , departure
    if ops == 'arrival':
        Data_raw = Data_raw.drop('ADR', axis=1)
        datelist = pd.DataFrame(pd.date_range('2019-01-01', '2019-12-31 23:00', freq = '1h'))
        test_result = pd.DataFrame({'Date' : datelist[start:end][0],
                                    'EAD' : Data_raw['EAD'][start:end],
                                    'Actual AAR' : Data_raw['AAR'][start:end], 
                                    'Predicted AAR' : reg_arrival.predict(Data_raw.drop('AAR', axis=1)[start:end]), 
                                    'Difference' : reg_arrival.predict(Data_raw.drop('AAR', axis=1)[start:end]) - Data_raw['AAR'][start:end]})
    elif ops == 'departure':
        Data_raw = Data_raw.drop('AAR', axis=1)        
        datelist = pd.DataFrame(pd.date_range('2019-01-01', '2019-12-31 23:00', freq = '1h'))
        test_result = pd.DataFrame({'Date' : datelist[start:end][0],
                                    'EDD' : Data_raw['EDD'][start:end],
                                    'Actual ADR' : Data_raw['ADR'][start:end], 
                                    'Predicted ADR' : reg_departure.predict(Data_raw.drop('ADR', axis=1)[start:end]), 
                                    'Difference' : reg_departure.predict(Data_raw.drop('ADR', axis=1)[start:end]) - Data_raw['ADR'][start:end]})
        
    return test_result

***
## Arrival

In [19]:
# Predict & Evaluation

y_hat_a = reg_arrival.predict(X_a)
print(f'RMSE : {np.sqrt(mean_squared_error(y_a, y_hat_a)):.4f}')
print(f'Training R^2 : {r2_score(y_train_a, reg_arrival.predict(X_train_a)) * 100:.4f}')
print(f'Test R^2 : {r2_score(y_test_a, reg_arrival.predict(X_test_a)) * 100:.4f}')

#print(f'explained variance score : {explained_variance_score(y_a, y_hat_a) :.4f}')
#print(f'max error : {max_error(y_a, y_hat_a) :.4f}')
#print(f'mean absolute error : {mean_absolute_error(y_a, y_hat_a) :.4f}')
#print(f'mean absolute percentage error : {mean_absolute_percentage_error(y_a, y_hat_a) :.4f}')
#print(f'mean squared log error : {mean_squared_log_error(y_a, y_hat_a) :.4f}')
#print(f'median absolute error : {median_absolute_error(y_a, y_hat_a) :.4f}')


RMSE : 2.0324
Training R^2 : 98.2838
Test R^2 : 93.9693


In [ ]:
# Feature Importance
imp = pd.DataFrame({'feature': Data_m.columns, 'importance': reg_arrival.feature_importances_})
imp = imp.sort_values('importance').set_index('feature')
imp.plot(kind='barh', figsize = (20,20))
plt.legend(loc='lower right')

In [ ]:
plot_metric(reg_arrival, figsize=(10,10))

In [ ]:
# AAR - pred
pred_test_a = Data_raw.drop('ADR', axis=1)
pred_test_a['pred_a'] = reg_arrival.predict(pred_test_a.drop('AAR', axis=1))
sns.pairplot(data=pred_test_a, vars=['AAR','pred_a'], size=10, plot_kws={'alpha': .5}, kind ='scatter' )

In [ ]:
# Test
Test_all = Test(Data_raw, ops='arrival', start = 0, end = 8760).sort_values('Difference')
Test_all['Difference'].abs().sort_values(ascending=False).head(30)

In [17]:
Test(Data_raw, ops='arrival', start = 0, end = 8760)

,Date,EAD,Actual AAR,Predicted AAR,Difference
0,2019-01-01 00:00:00,3.0,4.0,4.727593,0.727593
1,2019-01-01 01:00:00,3.0,2.0,2.080407,0.080407
2,2019-01-01 02:00:00,2.0,0.0,1.481987,1.481987
3,2019-01-01 03:00:00,1.0,6.0,5.421175,-0.578825
4,2019-01-01 04:00:00,15.0,20.0,20.148826,0.148826
...,...,...,...,...,...
8755,2019-12-31 19:00:00,28.0,26.0,28.454332,2.454332
8756,2019-12-31 20:00:00,29.0,39.0,35.687880,-3.312120
8757,2019-12-31 21:00:00,28.0,23.0,23.759404,0.759404
8758,2019-12-31 22:00:00,24.0,20.0,20.440395,0.440395


-------------------
## Departure

In [ ]:
# Predict & Evaluation
print(f'RMSE : {np.sqrt(mean_squared_error(y_d, reg_departure.predict(X_d))):.4f}')
print(f'Training R^2 : {r2_score(y_train_d, reg_departure.predict(X_train_d)) * 100:.4f}')
print(f'Test R^2 : {r2_score(y_test_d, reg_departure.predict(X_test_d)) * 100:.4f}')

In [ ]:
# Feature Importance
imp = pd.DataFrame({'feature': Data_m.columns, 'importance': reg_departure.feature_importances_})
imp = imp.sort_values('importance').set_index('feature')
imp.plot(kind='barh', figsize = (20,20))
plt.legend(loc='lower right')

In [ ]:
plot_metric(reg_departure, figsize=(10,10))

In [ ]:
# ADR - pred
pred_test_d = Data_raw.drop('AAR', axis=1)
pred_test_d['pred_d'] = reg_departure.predict(pred_test_d.drop('ADR', axis=1))
sns.pairplot(data=pred_test_d, vars=['ADR','pred_d'], size=10, plot_kws={'alpha': .5})

In [ ]:
# Test
Test_all = Test(Data_raw, ops='departure', start = 0, end = 8760).sort_values('Difference')
Test_all['Difference'].abs().sort_values(ascending=False).head(30)

***
***
***

# Maximum Capacity

In [ ]:
"""

# 예전 버전 : 원래 demand부터 시작 -> extra demand를 더함

def max_capacity(Data_raw, Data_m, example):   
    
    # extra
    extra_demand = 50
    
    # arrival
    data_a = Data_m.to_numpy()[example:example+1]
    XX_a = np.zeros((1,len(data_a.T)))
    for i in range(extra_demand+1):
        XX_a = np.append(XX_a, data_a, axis = 0)
        XX_a[i,0] = XX_a[i,0] + i-1
    XXX_a = XX_a[1:extra_demand+1]
    XXXX_a = np.arange(XXX_a[0,0], XXX_a[extra_demand-1,0]+1, 1)
    YYYY_a = reg_arrival.predict(XXX_a[0:extra_demand+1])
    max_aar = float(max(YYYY_a))
    actual_aar = int(Data_raw['AAR'][example:example+1])
    ead = int(Data_raw['EAD'][example:example+1])
    prediction_a = float(reg_arrival.predict(Data_m[example:example+1]))
    
    # departure
    data_d = Data_m.to_numpy()[example:example+1]
    XX_d = np.zeros((1,len(data_d.T)))
    for i in range(extra_demand+1):
        XX_d = np.append(XX_d, data_d, axis = 0)
        XX_d[i,2] = XX_d[i,2] + i-1
    XXX_d = XX_d[1:extra_demand+1]
    XXXX_d = np.arange(XXX_d[0,2], XXX_d[extra_demand-1,2]+1, 1)
    YYYY_d = reg_departure.predict(XXX_d[0:extra_demand+1])
    max_adr = float(max(YYYY_d))
    actual_adr = int(Data_raw['ADR'][example:example+1])
    edd = int(Data_raw['EDD'][example:example+1])
    prediction_d = float(reg_departure.predict(Data_m[example:example+1])) 

    # Total
    result_ = pd.DataFrame({'EAD' : XXXX_a ,'Predicted AAR' : YYYY_a, 'EDD' : XXXX_d ,'Predicted ADR' : YYYY_d, 
                            'Demand' : XXXX_a + YYYY_a, 'Capacity' : YYYY_a + YYYY_d})
    
    # plot
    plt.figure(figsize=(15,15))
    plt.title('Maximum Capacity', fontsize=30)
    plt.xlabel('Estimated Demands', fontsize=25)
    plt.ylabel('Capacity', fontsize=25)
    plt.plot(XXXX_a, YYYY_a, label = 'Arrival')
    plt.plot(XXXX_d, YYYY_d, label = 'Departure')
    plt.plot(XXXX_a[:-15]+XXXX_d[:-15], YYYY_a[:-15] + YYYY_d[:-15], label = 'Total')   # 안 예뻐서 뒤에 15개 자름
    plt.legend(prop={'size': 20})
    ax = plt.subplot()
    plt.text(0.95, 0.04,    # 위치조정
             f"""
             * Predicted Max AAR: {max_aar:.3f}\n
               Predicted AAR : {prediction_a:.3f}\n
               Actual AAR : {actual_aar} \n               
               EAD : {ead} \n\n
             
             * Predicted Max ADR: {max_adr:.3f}\n
               Predicted ADR : {prediction_d:.3f}\n
               Actual ADR : {actual_adr} \n               
               EDD : {edd} \n\n
             
             * Predicted Max Capacity: {max_aar + max_adr:.3f}\n
               Predicted Rate : {prediction_a + prediction_d:.3f}\n
               Actual Rate : {actual_aar + actual_adr} \n               
               Demand : {ead + edd}'
             """,
             fontsize=15, style='italic', transform=ax.transAxes, bbox={'facecolor': 'grey', 'alpha': 0, 'pad': 5})
    plt.show()    
    return result_
    
"""

In [ ]:
# *** training data에는 AAR, ADR 둘다 없어야 함
# 0부터 시작
# max capacity 선 추가
# 원래 demand에 대한 predict 점 추가

def max_capacity(Data_raw, Data_m, example):    # 50까지 늘림
    
    # extra
    demand = 80
    

    # arrival
    data_a = Data_m.to_numpy()[example:example+1]
    XX_a = np.zeros((1,len(data_a.T)))
    
    original_demand_a = int(data_a[0][0])
    for i in range(0,demand+1):
        XX_a = np.append(XX_a, data_a, axis = 0)
        XX_a[i,0] = XX_a[i,0] + i - original_demand_a
    
    XX_a[0] = XX_a[1]   
    XX_a[0,0] = 0   
    XXX_a = XX_a[0:demand+1]
    XXXX_a = np.arange(XXX_a[0,0], XXX_a[demand,0]+1, 1)
    YYYY_a = reg_arrival.predict(XXX_a[0:demand+1])
    max_aar = float(max(YYYY_a))
    actual_aar = int(Data_raw.drop('ADR', axis=1)['AAR'][example:example+1])
    ead = int(Data_raw.drop('ADR', axis=1)['EAD'][example:example+1])
    prediction_a = float(reg_arrival.predict(Data_m[example:example+1]))
    
    
    
    # departure
    data_d = Data_m.to_numpy()[example:example+1]
    XX_d = np.zeros((1,len(data_d.T)))
    
    original_demand_d = int(data_d[0][1])
    for i in range(0,demand+1):
        XX_d = np.append(XX_d, data_d, axis = 0)
        XX_d[i,1] = XX_d[i,1] + i - original_demand_d
        
    XX_d[0] = XX_d[1]           
    XX_d[0,1] = 0
    XXX_d = XX_d[0:demand+1]
    XXXX_d = np.arange(XXX_d[0,1], XXX_d[demand,1]+1, 1)
    YYYY_d = reg_departure.predict(XXX_d[0:demand+1])
    max_adr = float(max(YYYY_d))
    actual_adr = int(Data_raw.drop('AAR', axis=1)['ADR'][example:example+1])
    edd = int(Data_raw.drop('AAR', axis=1)['EDD'][example:example+1])
    prediction_d = float(reg_departure.predict(Data_m[example:example+1])) 

    
    
    # capacity - arrival과 departure을 수요의 비율로 늘렸을 때
    max_cap = int(max(Data_raw['AAR'] + Data_raw['ADR']))
    max_capacity = np.zeros(demand+1)
    for i in range(0,demand*2+1,2):
        if round(i*data_a[0,0]/(data_a[0,0]+data_d[0,1])) >=75:
            capa_arr = YYYY_a[75]
        elif round(i*data_d[0,1]/(data_a[0,0]+data_d[0,1])) >=75:
            capa_dep = YYYY_d[75]
        else:
            capa_arr = YYYY_a[round(i*data_a[0,0]/(data_a[0,0]+data_d[0,1]))]
            capa_dep = YYYY_d[round(i*data_d[0,1]/(data_a[0,0]+data_d[0,1]))]
        max_capacity[int(i/2)] = capa_arr + capa_dep
    
    
    
    # plot
    plt.figure(figsize=(15,15))
    plt.title('Maximum Capacity', fontsize=30)
    plt.xlabel('Estimated Demands', fontsize=25)
    plt.ylabel('Capacity', fontsize=25)
    ax = plt.subplot()

    plt.plot(XXXX_a, YYYY_a, label = 'Arrival')    # Arrival
    plt.plot(XXXX_d, YYYY_d, label = 'Departure')    # Departure
    plt.plot(XXXX_a[:-5] + XXXX_d[:-5], max_capacity[:-5], label = 'Total')    # Capacity - 안 예뻐서 뒤에 55개 자름
    plt.plot(XXXX_a[:-5]+XXXX_d[:-5], [max_cap]*(demand-4), label = f'Empirical Maximum = {max_cap}')    # 데이터 상 max capacity
    
    plt.plot(data_a[0,0], prediction_a,'ob', markersize = 10)    # 원래 arrival 예측값
    plt.plot(data_d[0,1], prediction_d,'or', markersize = 10)    # 원래 departure 예측값
    plt.plot(data_a[0,0]+data_d[0,1], prediction_a + prediction_d,'yo', markersize = 15)    # 원래 capacity 예측값
    plt.legend(prop={'size': 20}, loc = 'upper left')

    plt.text(0.95, 0.04,    # 위치조정
             f"""
             * Predicted Max AAR: {max_aar:.3f}\n
               Predicted AAR : {prediction_a:.3f}\n
               Actual AAR : {actual_aar} \n               
               EAD : {ead} \n\n
             
             * Predicted Max ADR: {max_adr:.3f}\n
               Predicted ADR : {prediction_d:.3f}\n
               Actual ADR : {actual_adr} \n               
               EDD : {edd} \n\n
             
             * Predicted Max Capacity: {max_aar + max_adr:.3f}\n
               Predicted Rate : {prediction_a + prediction_d:.3f}\n
               Actual Rate : {actual_aar + actual_adr} \n               
               Demand : {ead + edd}'
             """,
             fontsize=15, style='italic', transform=ax.transAxes, bbox={'facecolor': 'grey', 'alpha': 0, 'pad': 5})
    
    plt.show()   
    
    return Data_raw[example:example+1]   #result_

In [ ]:
# 원하는 시간의 Max Capacity 그래프

max_capacity(Data_raw, Data_m, 150)     # 숫자에 원하는 Data index(0-8760) 넣기

< Notable Samples> 

* pd.DataFrame(Data_raw).idxmax()             # 각 column별 max값의 index

AAR 최소 : 2 <br>
AAR 최대 : 1481 <br>
EAD 최대 : 641 <br>
ADR 최소 : 28 <br>
ADR 최대 : 538 <br>
EDD 최대 : 273 <br>
Total Rate(AAR+ADR) 최대(77) : 5703 <br>
Total Demand(EAD+EDD) 최대(70) : 4814 <br><br>

Arrival_reaminder 최대 : 5992 <br>
Departure_remainder 최대 : 8242 <br><br>

WSPD 최대 : 5990 <br>
W_GST 최대 : 5990 <br>
시정 최소 : 1755 <br>
Ceiling 최소 : 321 <br>
RVR 최소 1448 <br><br>

TAF WSPD 최대 : 5988 <br>
TAF W_GST 최대 : 5988 <br>
TAF 시정 최소 : 319 <br>
TAF Ceiling 최소 : 10?? <br>

In [ ]:
# Data : 불러온 파일
# Data_temp : 일단 필요 없어 보이는 column을 버린 것
# data_taf : dictinoary 안에 Data_6 - Data_24 전부 넣은 것
# Data_6, Data_12 .. : 해당 시간 전의 TAF만 있고, 나머지 시간의 TAF는 버린 것
# Data_raw : Data_#과 동일
# Data_m : Data_# 에서 AAR, ADR 뺀 것
# Data_a : Data_m 로 바꿈
# Data_d : Data_m 로 바꿈

***
***
***

# 추가 실험

##  Distribution Estimator

In [ ]:
# validation set = 30%
    
# Arrival

In [21]:
difference_val = reg_arrival.predict(X_val_a) - y_val_a    # validation set에서 예측값과 실제 값을 차이를 

In [26]:
difference_val

array([ 4.68034614, -0.77506686, -5.88151544, ..., -2.8753346 ,
       10.77832216,  3.74972198])

In [ ]:
# Hyperparameters
params = {'boosting_type' : 'gbdt',            # 'dart' 는 계산시간 길어짐, early stopping X / 'rf’ : Random Forest
          'metric': 'mse',
          'num_leaves' : 127,                  ## Maximum tree leaves for base learners (31)
          'max_depth' : - 1,                   # Maximum tree depth for base learners, <=0 means no limit (-1)
          'learning_rate' : 0.001,             ## Boosting learning rate (0.1)
          'n_estimators' : 10000000,           # Number of boosted trees to fit (100) -> fit에서 early stopping으로 제한해서 크게 설정함
          'subsample_for_bin' : 200000,        # Number of samples for constructing bins (200000)
          'objective' : 'regression',          # learning task and the corresponding learning objective (None)
          'class_weight' : None,               # * Use this parameter only for multi-class classification task
          'min_split_gain' : 0.0,              # Minimum loss reduction required to make a further partition on a leaf node of the tree (0)
          'min_child_weight' : 0.001,          # Minimum sum of instance weight (hessian) needed in a child (leaf) (0.001)
          'min_child_samples' : 1,             # Minimum number of data needed in a child (leaf) (20) - 마지막노드(리프)에 최소 몇가지 샘플이 있어야 하는지 
          'subsample' : 0.8,                   ## Subsample ratio of the training instance (1.0) - 개별 트리를 학습시키는데 몇 %의 데이터를 사용할 것 인지, row sampling
          'subsample_freq' : 1,                # Frequency of subsample, <=0 means no enable (0) - 몇개의 트리마다 subsampling을 할 것인지
          'colsample_bytree' : 0.8,            ## Subsample ratio of columns when constructing each tree (1.0) - 몇 %의 column을 sampling 할 것인지
          'reg_alpha' : 0.0,                   # L1 regularization term on weights (0)
          'reg_lambda' : 0.0,                  # L2 regularization term on weights (0)
          'random_state' : seed,               # Random number seed (None)
          'n_jobs' : - 1,                      # Number of parallel threads (-1) - 몇 개의 병렬작업을 할 것인지 (-1 = 모든 가능한 것 전부)
          'silent' : True,                     # Whether to print messages while running boosting (True)
          'importance_type' : 'split'}         # ‘split’: result contains numbers of times the feature is used in a model
                                               # ‘gain’ : result contains total gains of splits which use the feature

# 최적 hyperparameter 찾기!!

In [ ]:
# Model fitting
distribution_arrival = LGBMRegressor(**params_a)

distribution_arrival.fit(X_train_a, y_train_a,
                sample_weight = None,                   # Weights of training data
                init_score = None,                      # Weights of training data
                eval_sample_weight = None,              # Weights of eval data
                eval_init_score = None,                 # Init score of eval data.   
                eval_metric = 'l2',                     # Default: ‘l2’ for LGBMRegressor, ‘logloss’ for LGBMClassifier
                early_stopping_rounds = 10,             # loss fuction이 n번 이상 좋아지지 않으면 멈춰라
                verbose = True)            

***
***
***

# 모델 저장

In [ ]:
import joblib

lgbr_a = "../evaluate/lgbr_a.pkl"
lgbr_d = "../evaluate/lgbr_d.pkl"
joblib.dump(reg_arrival, lgbr_a)
joblib.dump(reg_departure, lgbr_d)